In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Hyperparameters
latent_dim = 100
image_size = 224
batch_size = 128
epochs = 300
learning_rate = 0.0002

In [2]:
# Generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 7 * 7 * 256),
            nn.BatchNorm1d(7 * 7 * 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Unflatten(1, (256, 7, 7)),
            nn.ConvTranspose2d(256, 128, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(64, 32, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(32, 16, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(16, 3, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)


In [3]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.4),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.4),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.4),
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.4),
            nn.Flatten(),
            nn.Linear(512 * 14 * 14, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [4]:
def create_fake_data(generator, num_samples):
    input_data = torch.randn(num_samples, latent_dim, device=device)
    fake_images = generator(input_data)
    return fake_images


In [5]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5] * 3, [0.5] * 3)
])

dataset = datasets.ImageFolder(r'C:\GANproject\dataset\archive', transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [6]:
'''def show_generated_images(fake_images, epoch):
    fake_images = (fake_images + 1) / 2.0  # Rescale from [-1, 1] to [0, 1]
    save_image(fake_images[:10], f'{epoch}_generated_images.png', nrow=5, normalize=True)'''
def show_generated_images(fake_images, epoch, save_dir=r'C:\GANproject\Genimages'):
    if not os.path.exists(r'C:\GANproject\Genimages'):
        os.makedirs(r'C:\GANproject\Genimages')
    fake_images = (fake_images + 1) / 2.0  # Rescale from [-1, 1] to [0, 1]
    save_image(fake_images[:10], os.path.join(save_dir, f'{epoch}_generated_images.png'), nrow=5, normalize=True)



In [7]:
def save_model(generator, epoch):
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_path = f'generator_model_epoch_{epoch}_{now}.pth'
    torch.save(generator.state_dict(), model_path)
    print(f"Model saved to {model_path}")

In [8]:
def train(generator, discriminator, dataloader, optimizer_g, optimizer_d, criterion, epochs):
    for epoch in range(epochs):
        for i, (real_images, _) in enumerate(dataloader):
            real_images = real_images.to(device)
            current_batch_size = real_images.size(0)

            # Train Discriminator
            optimizer_d.zero_grad()
            fake_images = create_fake_data(generator, current_batch_size)
            real_labels = torch.ones(current_batch_size, 1, device=device)
            fake_labels = torch.zeros(current_batch_size, 1, device=device)

            output_real = discriminator(real_images)
            output_fake = discriminator(fake_images.detach())

            loss_real = criterion(output_real, real_labels)
            loss_fake = criterion(output_fake, fake_labels)

            loss_d = loss_real + loss_fake
            loss_d.backward()
            optimizer_d.step()

            # Train Generator
            optimizer_g.zero_grad()
            output_fake = discriminator(fake_images)
            loss_g = criterion(output_fake, real_labels)
            loss_g.backward()
            optimizer_g.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}] | Loss D: {loss_d.item()} | Loss G: {loss_g.item()}')
            show_generated_images(fake_images, epoch)
            save_model(generator, epoch)

In [9]:
# Instantiate models and optimizers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Check if models are on GPU
print(f"Generator is on: {next(generator.parameters()).device}")
print(f"Discriminator is on: {next(discriminator.parameters()).device}")


Generator is on: cuda:0
Discriminator is on: cuda:0


In [10]:
# Instantiate models and optimizers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = Generator().to(device)
discriminator = Discriminator().to(device)
optimizer_g = optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
criterion = nn.BCELoss()

# Train the models
train(generator, discriminator, dataloader, optimizer_g, optimizer_d, criterion, epochs)

C:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [10/300] | Loss D: 0.667374849319458 | Loss G: 1.9894335269927979
Model saved to generator_model_epoch_9_20240903_025734.pth
Epoch [20/300] | Loss D: 1.6684943437576294 | Loss G: 0.5670934915542603
Model saved to generator_model_epoch_19_20240903_033039.pth
Epoch [30/300] | Loss D: 0.8166041374206543 | Loss G: 2.3505280017852783
Model saved to generator_model_epoch_29_20240903_040344.pth
Epoch [40/300] | Loss D: 0.5800735950469971 | Loss G: 2.043039321899414
Model saved to generator_model_epoch_39_20240903_043648.pth
Epoch [50/300] | Loss D: 0.5109657049179077 | Loss G: 2.3526721000671387
Model saved to generator_model_epoch_49_20240903_050953.pth
Epoch [60/300] | Loss D: 0.9265035390853882 | Loss G: 2.0039336681365967
Model saved to generator_model_epoch_59_20240903_054258.pth
Epoch [70/300] | Loss D: 0.4779430031776428 | Loss G: 2.7353222370147705
Model saved to generator_model_epoch_69_20240903_061602.pth
Epoch [80/300] | Loss D: 0.4511052668094635 | Loss G: 2.50262188911438
M